In [ ]:
import pandas as pd
from xgboost import XGBRegressor
from scipy.stats.mstats import winsorize

# Step 1: Load and prepare the dataset
df = pd.read_csv("data_final.csv", parse_dates=["datum"])
df = df.sort_values(["volgnr", "datum"]).reset_index(drop=True)

# Step 2: Create date-based features and define the target (next month's cash flow)
df["jaar"] = df["datum"].dt.year
df["maand"] = df["datum"].dt.month
df["target"] = df.groupby("volgnr")["totale_kasstroom"].shift(-1)

# Step 3: Drop rows with missing target values
df = df.dropna(subset=["target"])

# Step 4: Winsorize the target to limit the effect of extreme outliers
df["target"] = winsorize(df["target"], limits=[0.01, 0.01])

# Step 5: Feature engineering - create lags and derived ratios
df["eindsaldo_liquide_middelen_lag_1"] = df.groupby("volgnr")["eindsaldo_liquide_middelen"].shift(1)
df["mutaties_vorderingen_en_schulden_lag_1"] = df.groupby("volgnr")["mutaties_vorderingen_en_schulden"].shift(1)
df["eindsaldo_liquide_middelen_lag_6"] = df.groupby("volgnr")["eindsaldo_liquide_middelen"].shift(6)
df["mutaties_lag_6"] = df.groupby("volgnr")["mutaties_vorderingen_en_schulden"].shift(6)
df["ratio_schulden_opbrengst"] = df["mutaties_vorderingen_en_schulden"] / (df["totaal_opbrengsten_lag_1"] + 1e-6)
df["kasratio"] = df["kas"] / (df["totale_kasstroom_lag_1"] + 1e-6)
df["melkprijs_diff_6"] = df["melkprijs_per_kg"] - df["melkprijs_per_kg"].shift(6)

# Step 6: Drop any remaining rows with missing values
df = df.dropna()

# Step 7: Select numeric features for modeling
exclude = ["datum", "volgnr", "boekjaar", "jaar", "target", "set"]
features = [col for col in df.select_dtypes(include="number").columns if col not in exclude]
X = df[features]
y = df["target"]

# Step 8: Train the XGBoost regression model
model = XGBRegressor(
    learning_rate=0.1,
    max_depth=3,
    n_estimators=100,
    subsample=1.0,
    colsample_bytree=1.0,
    random_state=42,
    eval_metric="rmse",
    use_label_encoder=False
)
model.fit(X, y)

# Step 9: Save the trained model and feature matrix for SHAP or further analysis
model.save_model("xgb_model.json")
X.to_csv("X_for_shap.csv", index=False)


In [ ]:
import shap
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
import numpy as np

# Step 1: Load the trained model and feature matrix
model = xgb.XGBRegressor()
model.load_model("xgb_model.json")
X = pd.read_csv("X_for_shap.csv")

# Step 2: Create a SHAP explainer for the model
explainer = shap.Explainer(model)
shap_values = explainer(X)

# Step 3: Compute mean absolute SHAP values per feature
top_n = 30
shap_importance = np.abs(shap_values.values).mean(axis=0)

# Step 4: Create a DataFrame with feature names and importance
shap_df = pd.DataFrame({
    "feature": X.columns,
    "mean_abs_shap": shap_importance
}).sort_values(by="mean_abs_shap", ascending=False)

# Step 5: Display top N most influential features
print(f"\nTop {top_n} SHAP features:")
print(shap_df.head(top_n)[["feature"]].to_string(index=False))


In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from scipy.stats.mstats import winsorize
from itertools import product

# Step 1: Load dataset
df = pd.read_csv("data_final.csv", parse_dates=["datum"])
df = df.sort_values(["volgnr", "datum"]).reset_index(drop=True)

# Step 2: Add time-based features
df["jaar"] = df["datum"].dt.year
df["maand"] = df["datum"].dt.month

# Step 3: Define target variable (next month's cash flow)
df["target"] = df.groupby("volgnr")["totale_kasstroom"].shift(-1)
df = df.dropna(subset=["target"])
df["target"] = pd.Series(winsorize(df["target"], limits=[0.01, 0.01]), index=df.index)

# Step 4: Create lagged and derived features
df["eindsaldo_liquide_middelen_lag_1"] = df.groupby("volgnr")["eindsaldo_liquide_middelen"].shift(1)
df["mutaties_vorderingen_en_schulden_lag_1"] = df.groupby("volgnr")["mutaties_vorderingen_en_schulden"].shift(1)
df["melkprijs_diff_6"] = df["melkprijs_per_kg"] - df["melkprijs_per_kg"].shift(6)

# Step 5: Create train/val/test split based on farms
farms = df["volgnr"].unique()
trainval_farms, test_farms = train_test_split(farms, test_size=0.2, random_state=42)
train_farms, val_farms = train_test_split(trainval_farms, test_size=0.2, random_state=42)

df["set"] = "none"
df.loc[df["volgnr"].isin(train_farms) & (df["jaar"] < 2024), "set"] = "train"
df.loc[df["volgnr"].isin(val_farms) & (df["jaar"] < 2024), "set"] = "val"
df.loc[df["volgnr"].isin(test_farms) & (df["jaar"] == 2024), "set"] = "test"
df = df[df["set"] != "none"]

# Step 6: Select features
features = [col for col in df.columns if col not in [
    "datum", "jaar", "target", "set", "totale_kasstroom"
]]

# Step 7: Prepare datasets
X_train = df[df["set"] == "train"][features]
y_train = df[df["set"] == "train"]["target"]
X_val = df[df["set"] == "val"][features]
y_val = df[df["set"] == "val"]["target"]
X_test = df[df["set"] == "test"][features]
y_test = df[df["set"] == "test"]["target"]

# Step 8: Manual grid search on train+val
param_grid = {
    "learning_rate": [0.009, 0.01, 0.05],
    "max_depth": [4, 5, 6],
    "n_estimators": [800, 1000, 1100]
}

best_rmse = float("inf")
best_model = None
best_params = {}

print("\nStarting grid search...\n")
for lr, md, ne in product(param_grid["learning_rate"], param_grid["max_depth"], param_grid["n_estimators"]):
    model = XGBRegressor(
        learning_rate=lr,
        max_depth=md,
        n_estimators=ne,
        subsample=0.8,
        colsample_bytree=1.0,
        random_state=42,
        eval_metric="rmse",
        use_label_encoder=False
    )
    model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
    y_val_pred = model.predict(X_val)
    rmse = mean_squared_error(y_val, y_val_pred, squared=False)

    print(f"lr={lr}, max_depth={md}, n_estimators={ne} → Validation RMSE: €{rmse:,.2f}")
    if rmse < best_rmse:
        best_rmse = rmse
        best_model = model
        best_params = {"learning_rate": lr, "max_depth": md, "n_estimators": ne}

print("\nBest Hyperparameters:")
print(best_params)

# Step 9: Retrain model on full training + validation data
X_trainval = pd.concat([X_train, X_val])
y_trainval = pd.concat([y_train, y_val])

final_model = XGBRegressor(
    **best_params,
    subsample=0.8,
    colsample_bytree=1.0,
    random_state=42,
    eval_metric="rmse",
    use_label_encoder=False
)
final_model.fit(X_trainval, y_trainval)

# Step 10: Predictions
y_trainval_pred = final_model.predict(X_trainval)
y_val_pred = final_model.predict(X_val)
y_test_pred = final_model.predict(X_test)

# Step 11: Evaluation function
def evaluate(y_true, y_pred, label):
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    return {
        "set": label,
        "RMSE": rmse,
        "MAE": mae,
        "R2": r2
    }

# Step 12: Collect and print results
results = [
    evaluate(y_trainval, y_trainval_pred, "Train+Val"),
    evaluate(y_val, y_val_pred, "Validation"),
    evaluate(y_test, y_test_pred, "Test")
]

results_df = pd.DataFrame(results)
print("\nXGBoost Results Summary:")
print(results_df.round(3))


In [ ]:
import matplotlib.pyplot as plt

# Step 11: Combine historical data and test set predictions
df_all = df.copy()
df_all["y_true"] = np.nan
df_all["y_pred"] = np.nan

# Assign actual values
df_all.loc[df["set"] == "train", "y_true"] = y_train.values
df_all.loc[df["set"] == "val", "y_true"] = y_val.values
df_all.loc[df["set"] == "test", "y_true"] = y_test.values
df_all.loc[df["set"] == "test", "y_pred"] = y_test_pred

# Set datetime index and resample to monthly averages
df_all = df_all.set_index("datum").sort_index()
monthly_avg = df_all.resample("M")[["y_true", "y_pred"]].mean()

# Step 12: Plot actual vs. forecasted cash flow
plt.figure(figsize=(12, 5))
plt.plot(
    monthly_avg.index,
    monthly_avg["y_true"],
    label="Actual",
    color="blue",
    linewidth=2
)
plt.plot(
    monthly_avg.index,
    monthly_avg["y_pred"],
    label="Forecast (2024)",
    color="orangered",
    linestyle="--",
    linewidth=2
)

plt.title("XGBoost Forecast – Monthly Average Cash Flow (2020–2024)")
plt.xlabel("Date")
plt.ylabel("Average Cash Flow (€)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import shap
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Create SHAP explainer and compute SHAP values
explainer = shap.Explainer(model_final, X_trainval)
shap_values = explainer(X_trainval)

# Calculate mean absolute SHAP values per feature
shap_df = pd.DataFrame(
    np.abs(shap_values.values).mean(axis=0),
    index=X_trainval.columns,
    columns=["mean_abs_shap"]
)

# Select top 10 most important features
shap_df = shap_df.sort_values("mean_abs_shap", ascending=False).head(10)

# Create horizontal bar plot
plt.figure(figsize=(10, 5))
plt.barh(shap_df.index[::-1], shap_df["mean_abs_shap"][::-1])
plt.xlabel("Mean |SHAP value|")
plt.title("Top 10 SHAP Feature Importances (XGBoost)")
plt.grid(True, axis='x', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


In [ ]:
# Step 1: Define mapping from Dutch to English feature names
feature_translation = {
    "eindsaldo_liquide_middelen": "Ending Cash Balance",
    "mutaties_vorderingen_en_schulden": "Mutations in Receivables and Debts",
    "crediteuren": "Creditors",
    "totale_kasstroom": "Total Cash Flow",
    "maand": "Month",
    "eindsaldo_liquide_middelen_lag_6": "Ending Cash Balance (lag 6)",
    "mutaties_vorderingen_en_schulden_lag_1": "Mutations Receivables/Debts (lag 1)",
    "melkprijs_per_kg": "Milk Price per kg",
    "overige_vorderingen": "Other Receivables",
    "melkprijs_diff_6": "Milk Price Difference (6m)"
}

# Step 2: Translate index labels (feature names) in SHAP DataFrame
shap_df.index = shap_df.index.map(lambda x: feature_translation.get(x, x))

# Step 3: Plot SHAP feature importance with translated labels
plt.figure(figsize=(10, 5))
plt.barh(shap_df.index[::-1], shap_df["mean_abs_shap"][::-1])
plt.xlabel("Mean |SHAP value|")
plt.title("Top 10 SHAP Feature Importances (XGBoost)")
plt.grid(True, axis='x', linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import pandas as pd

# Step 1: Evaluate model performance per farm cluster on the test set
results = []

# Ensure 'bedrijf_cluster' exists in the test data
for cluster_id in sorted(df[df["set"] == "test"]["bedrijf_cluster"].dropna().unique()):
    df_cluster = df[(df["set"] == "test") & (df["bedrijf_cluster"] == cluster_id)].copy()
    if df_cluster.empty:
        continue

    X_test_cluster = df_cluster[features]
    y_test_cluster = df_cluster["target"]
    y_pred_cluster = model_final.predict(X_test_cluster)

    # Compute evaluation metrics
    rmse = mean_squared_error(y_test_cluster, y_pred_cluster, squared=False)
    r2 = r2_score(y_test_cluster, y_pred_cluster)
    mae = mean_absolute_error(y_test_cluster, y_pred_cluster)

    results.append({
        "Cluster": int(cluster_id),
        "Test observations": len(y_test_cluster),
        "RMSE": round(rmse, 2),
        "R²": round(r2, 3),
        "MAE": round(mae, 2)
    })

# Display evaluation results per cluster
results_df = pd.DataFrame(results).sort_values("Cluster")
print("\nResults by Cluster:")
print(results_df.to_string(index=False))


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Step 2: Combine true and predicted values for train, val, and test sets
df_all = df.copy()
df_all["y_true"] = np.nan
df_all["y_pred"] = np.nan

df_all.loc[df["set"] == "train", "y_true"] = y_train.values
df_all.loc[df["set"] == "val", "y_true"] = y_val.values
df_all.loc[df["set"] == "test", "y_true"] = y_test.values
df_all.loc[df["set"] == "test", "y_pred"] = y_test_pred

df_all = df_all.set_index("datum").sort_index()

# Step 3: Get list of available clusters
clusters = sorted(df_all["bedrijf_cluster"].dropna().unique())

# Step : Create subplots, one per cluster
fig, axes = plt.subplots(1, len(clusters), figsize=(18, 5), sharey=True)  # 1 row, multiple columns

for ax, cluster_id in zip(axes, clusters):
    df_cluster = df_all[df_all["bedrijf_cluster"] == cluster_id].copy()
    if df_cluster.empty:
        continue

    # Compute monthly averages
    monthly_avg = df_cluster.resample("M")[["y_true", "y_pred"]].mean()

    # Plot actual and forecasted values
    ax.plot(monthly_avg.index, monthly_avg["y_true"], label="Actual", color="blue", linewidth=2)
    ax.plot(monthly_avg.index, monthly_avg["y_pred"], label="Forecast (2024)", color="orangered", linestyle="--", linewidth=2)

    # Vertical line for start of forecast period
    ax.axvline(pd.to_datetime("2024-01-01"), color="gray", linestyle=":", label="Forecast Start")

    # Axis styling
    ax.set_title(f"Cluster {int(cluster_id)}", fontsize=13)
    ax.set_xlabel("Date", fontsize=11)
    ax.grid(True, linestyle="--", alpha=0.6)
    ax.set_xticks(pd.date_range(start="2020-01-01", end="2024-12-01", freq="YS"))
    ax.tick_params(axis='x', rotation=45)

# Only the first plot gets the Y label
axes[0].set_ylabel("Average Cash Flow (€)", fontsize=12)

# Shared legend
handles, labels = ax.get_legend_handles_labels()
fig.legend(handles, labels, loc="upper center", ncol=3, fontsize=11)

# Figure title and layout adjustment
fig.suptitle("Average Monthly Cash Flow per Cluster (2020–2024)", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.95])  # Leave room for the main title
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from scipy.stats.mstats import winsorize

# Step 1: Load and sort the dataset
df = pd.read_csv("data_final.csv", parse_dates=["datum"])
df = df.sort_values(["volgnr", "datum"]).reset_index(drop=True)

# Step 2: Time-based features
df["jaar"] = df["datum"].dt.year
df["maand"] = df["datum"].dt.month

# Step 3: Define target (next month's cash flow)
df["target"] = df.groupby("volgnr")["totale_kasstroom"].shift(-1)
df = df.dropna(subset=["target"])
df["target"] = pd.Series(winsorize(df["target"], limits=[0.01, 0.01]), index=df.index)

# Step 4: Additional lagged and derived features
df["eindsaldo_liquide_middelen_lag_1"] = df.groupby("volgnr")["eindsaldo_liquide_middelen"].shift(1)
df["mutaties_vorderingen_en_schulden_lag_1"] = df.groupby("volgnr")["mutaties_vorderingen_en_schulden"].shift(1)
df["eindsaldo_liquide_middelen_lag_6"] = df.groupby("volgnr")["eindsaldo_liquide_middelen"].shift(6)
df["mutaties_lag_6"] = df.groupby("volgnr")["mutaties_vorderingen_en_schulden"].shift(6)
df["ratio_schulden_opbrengst"] = df["mutaties_vorderingen_en_schulden"] / (df["totaal_opbrengsten_lag_1"] + 1e-6)
df["kasratio"] = df["kas"] / (df["totale_kasstroom_lag_1"] + 1e-6)
df["melkprijs_diff_6"] = df["melkprijs_per_kg"] - df["melkprijs_per_kg"].shift(6)

# Step 5: Split data by farms into train, validation, and test sets
farms = df["volgnr"].unique()
trainval_farms, test_farms = train_test_split(farms, test_size=0.2, random_state=42)
train_farms, val_farms = train_test_split(trainval_farms, test_size=0.2, random_state=42)

df["set"] = "none"
df.loc[df["volgnr"].isin(train_farms) & (df["jaar"] < 2024), "set"] = "train"
df.loc[df["volgnr"].isin(val_farms) & (df["jaar"] < 2024), "set"] = "val"
df.loc[df["volgnr"].isin(test_farms) & (df["jaar"] == 2024), "set"] = "test"
df = df[df["set"] != "none"]

# Step 6: Feature selection (excluding weather or external data)
all_features = [
    "eindsaldo_liquide_middelen",
    "crediteuren",
    "ratio_schulden_opbrengst",
    "totale_kasstroom",
    "mutaties_vorderingen_en_schulden",
    "maand",
    "melkprijs_diff_6",
    "eindsaldo_liquide_middelen_lag_6",
    "melkprijs_per_kg",
    "mutaties_vorderingen_en_schulden_lag_1",
    "resultaat_vóór_bijzondere_resultaten",
    "koesaldo_per_kg_fosfaat",
    "accountantskosten",
    "overige_vorderingen",
    "energiekosten",
    "totale_kasstroom_lag_1",
    "debiteuren",
    "marge",
    "omzetbelasting_(tussenrekening)",
    "financiële_baten_en_lasten",
    "verzekeringen",
    "melkkoeien_-_waarde_per_dier_eindbalans",
    "rentelasten_en_soortgelijke_kosten_(-)",
    "voorschot_melkgeld",
    "krachtvoerkosten",
    "mutatie_debiteuren",
    "kas"
]

features = [f for f in all_features if f in df.columns]

# Step 7: Create training, validation, and test datasets
X_train = df[df["set"] == "train"][features]
y_train = df[df["set"] == "train"]["target"]
X_val = df[df["set"] == "val"][features]
y_val = df[df["set"] == "val"]["target"]
X_trainval = pd.concat([X_train, X_val])
y_trainval = pd.concat([y_train, y_val])
X_test = df[df["set"] == "test"][features]
y_test = df[df["set"] == "test"]["target"]

# Step 8: Train initial model on training set
model_val = XGBRegressor(
    learning_rate=0.01,
    max_depth=4,
    n_estimators=800,
    subsample=0.8,
    colsample_bytree=1.0,
    random_state=42,
    eval_metric="rmse",
    use_label_encoder=False
)
model_val.fit(X_train, y_train)
y_val_pred = model_val.predict(X_val)

# Step 9: Final model trained on train + validation
model_final = XGBRegressor(
    learning_rate=0.01,
    max_depth=4,
    n_estimators=800,
    subsample=0.8,
    colsample_bytree=1.0,
    random_state=42,
    eval_metric="rmse",
    use_label_encoder=False
)
model_final.fit(X_trainval, y_trainval)

# Step 10: Predictions
y_trainval_pred = model_final.predict(X_trainval)
y_test_pred = model_final.predict(X_test)

# Step 11: Evaluation function
def evaluate(y_true, y_pred, label):
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    print(f"\n{label} Results:")
    print(f"RMSE: €{rmse:,.2f}")
    print(f"R²: {r2:.3f}")
    print(f"MAE: €{mae:,.2f}")
    return rmse, r2, mae

# Evaluate on all sets
evaluate(y_trainval, y_trainval_pred, "Train+Validation")
evaluate(y_val, y_val_pred, "Validation")
evaluate(y_test, y_test_pred, "Test")

# Step 12: Visualization – monthly average predictions on the test set
df_plot = df[df["set"] == "test"].copy()
df_plot["y_true"] = y_test.values
df_plot["y_pred"] = y_test_pred
df_plot["maand"] = df_plot["datum"].dt.month
monthly_avg = df_plot.groupby("maand")[["y_true", "y_pred"]].mean()

plt.figure(figsize=(10, 5))
plt.plot(monthly_avg.index, monthly_avg["y_true"], label="Actual", marker="o")
plt.plot(monthly_avg.index, monthly_avg["y_pred"], label="Predicted", marker="o", linestyle="--")
plt.title("XGB Forecast Without Exogenous Data – 2024 (Test Farms)")
plt.xlabel("Month")
plt.ylabel("Average Cash Flow (€)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Step 11: Merge actual and predicted values for all data splits
df_all = df.copy()
df_all["y_true"] = np.nan
df_all["y_pred"] = np.nan

df_all.loc[df["set"] == "train", "y_true"] = y_train.values
df_all.loc[df["set"] == "val", "y_true"] = y_val.values
df_all.loc[df["set"] == "test", "y_true"] = y_test.values
df_all.loc[df["set"] == "test", "y_pred"] = y_test_pred

# Resample to monthly average values
df_all = df_all.set_index("datum").sort_index()
monthly_avg = df_all.resample("M")[["y_true", "y_pred"]].mean()

# Step 12: Plot historical vs. forecasted monthly cash flow
plt.figure(figsize=(12, 5))
plt.plot(monthly_avg.index, monthly_avg["y_true"], label="Actual", color="blue", linewidth=2)
plt.plot(monthly_avg.index, monthly_avg["y_pred"], label="Forecast (2024)", color="orangered", linestyle="--", linewidth=2)
plt.axvline(pd.to_datetime("2024-01-01"), color="gray", linestyle=":", label="Forecast Start")

plt.title("XGBoost Forecast Without Exogenous Variables – Monthly Average Cash Flow (2020–2024)")
plt.xlabel("Date")
plt.ylabel("Average Cash Flow (€)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()
